In [1]:
!ls

adc.json  drive  sample_data


In [3]:
cd drive/Houston/

/content/drive/Houston


In [0]:
import numpy as np
import cv2

In [0]:
moment_lidar = []
for i in range(4):
    img_path = "moment/lidar_" + str(i) + ".png"
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    moment_lidar.append(img)
    
moment_lidar = np.array(moment_lidar)

In [17]:
print(moment_lidar.shape)

(4, 349, 1905)


In [0]:
train_file_name = "labels/train.txt"
test_file_name = "labels/test.txt"

In [0]:
file = open(train_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
train_array = np.array(triplets, dtype=int)
file.close()


file = open(test_file_name)
triplets = file.read().split()
for i in range(0, len(triplets)):
    triplets[i] = triplets[i].split(",")
test_array = np.array(triplets, dtype=int)
file.close()

HEIGHT = train_array.shape[0]
WIDTH = train_array.shape[1]


In [0]:
moment_lidar_train_data = []
moment_lidar_test_data = []
train_labels = []
test_labels = []
for i in range(HEIGHT):
    for j in range(WIDTH):
        if train_array[i, j] != 0:
            moment_lidar_train_data.append(moment_lidar[:, i, j])
            train_labels.append(train_array[i, j])
        if test_array[i,j] != 0:
            moment_lidar_test_data.append(moment_lidar[:, i, j])
            test_labels.append(test_array[i, j])

In [0]:
moment_lidar_train_data = np.array(moment_lidar_train_data)
moment_lidar_test_data = np.array(moment_lidar_test_data)
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [22]:
print(moment_lidar_train_data.shape)
print(moment_lidar_test_data.shape)
print(train_labels.shape)
print(test_labels.shape)

(2832, 4)
(12197, 4)
(2832,)
(12197,)


In [23]:
import keras

train_one_hot = keras.utils.to_categorical(train_labels-1)
test_one_hot = keras.utils.to_categorical(test_labels-1)

Using TensorFlow backend.


In [25]:
print(train_one_hot.shape)
print(test_one_hot.shape)

(2832, 15)
(12197, 15)


In [0]:
LiDAR_PATCH_SIZE = 41
CONV1 = 500
CONV2 = 100
FC1 = 200
FC2 = 84
LEARNING_RATE = 0.005
BATCH_SIZE = 25

In [0]:
import numpy as np
padded_moment_lidar = np.lib.pad(moment_lidar, ((0,0), (LiDAR_PATCH_SIZE//2, LiDAR_PATCH_SIZE//2), (LiDAR_PATCH_SIZE//2,LiDAR_PATCH_SIZE//2)), 'reflect')

In [28]:
print(padded_moment_lidar.shape)

(4, 389, 1945)


In [0]:
def get_patches(data, patch_size, row, column):
  offset = patch_size // 2
  row_low = row - offset
  row_high = row + offset
  col_low = column - offset
  col_high = column + offset
  return data[0:, row_low:row_high + 1, col_low:col_high + 1].reshape(patch_size, patch_size, data.shape[0])

In [0]:
moment_lidar_train_patches = []
moment_lidar_test_patches = []

for i in range(HEIGHT):
  for j in range(WIDTH):
    if train_array[i, j] != 0:
      moment_lidar_train_patches.append(get_patches(padded_moment_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))
    if test_array[i, j] != 0:
      moment_lidar_test_patches.append(get_patches(padded_moment_lidar, LiDAR_PATCH_SIZE, i+LiDAR_PATCH_SIZE//2, j+LiDAR_PATCH_SIZE//2))

In [0]:
moment_lidar_train_patches = np.array(moment_lidar_train_patches)
moment_lidar_test_patches = np.array(moment_lidar_test_patches)

In [0]:
print(moment_lidar_train_patches.shape)
print(moment_lidar_test_patches.shape)

In [33]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Conv2D, Dense, Flatten
from tensorflow.python.keras.layers import InputLayer
from tensorflow.python.keras.layers import  MaxPooling2D
from tensorflow.python.keras.layers import BatchNormalization, Dropout
from tensorflow.python.keras.optimizers import Adam,SGD

BANDS = moment_lidar_train_patches.shape[3]
NUM_CLS = train_one_hot.shape[1]


moment_lidar_model = Sequential()
moment_lidar_model.add(InputLayer(input_shape=(LiDAR_PATCH_SIZE, LiDAR_PATCH_SIZE, BANDS)))
moment_lidar_model.add(Conv2D(kernel_size=6, strides=2, filters=CONV1, padding='same', activation='relu', name='conv1'))
moment_lidar_model.add(BatchNormalization())
moment_lidar_model.add(MaxPooling2D(pool_size=2, strides=2))
moment_lidar_model.add(Conv2D(kernel_size=5, strides=2, filters=CONV2, padding='same', activation='relu', name='conv2'))
moment_lidar_model.add(BatchNormalization())
moment_lidar_model.add(MaxPooling2D(pool_size=2, strides=2))


moment_lidar_model.add(Flatten())
moment_lidar_model.add(Dense(FC1, activation='relu'))
moment_lidar_model.add(Dropout(0.7))
moment_lidar_model.add(Dense(FC2, activation='relu'))
moment_lidar_model.add(Dropout(0.5))
moment_lidar_model.add(Dense(NUM_CLS, activation='softmax'))

sgd = SGD(lr=LEARNING_RATE, decay=1e-6, momentum=0.9, nesterov=True)
moment_lidar_model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])


moment_lidar_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 21, 21, 500)       72500     
_________________________________________________________________
batch_normalization (BatchNo (None, 21, 21, 500)       2000      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 500)       0         
_________________________________________________________________
conv2 (Conv2D)               (None, 5, 5, 100)         1250100   
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 5, 100)         400       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 2, 2, 100)         0         
_________________________________________________________________
flatten (Flatten)            (None, 400)               0         
__________

In [57]:
history = moment_lidar_model.fit(moment_lidar_train_patches, train_one_hot, batch_size=BATCH_SIZE, shuffle=True, epochs=50)

Epoch 1/50
2832/2832 [==============================] - 3s 922us/step - loss: 0.0314 - acc: 0.9898
Epoch 2/50
2832/2832 [==============================] - 3s 886us/step - loss: 0.0383 - acc: 0.9880
Epoch 3/50
2832/2832 [==============================] - 2s 882us/step - loss: 0.0479 - acc: 0.9855
Epoch 4/50
2832/2832 [==============================] - 2s 875us/step - loss: 0.0386 - acc: 0.9852
Epoch 5/50
2832/2832 [==============================] - 2s 878us/step - loss: 0.0303 - acc: 0.9894
Epoch 6/50
2832/2832 [==============================] - 2s 879us/step - loss: 0.0391 - acc: 0.9883
Epoch 7/50
2832/2832 [==============================] - 3s 884us/step - loss: 0.0313 - acc: 0.9887
Epoch 8/50
2832/2832 [==============================] - 2s 876us/step - loss: 0.0343 - acc: 0.9922
Epoch 9/50
2832/2832 [==============================] - 2s 877us/step - loss: 0.0576 - acc: 0.9799
Epoch 10/50
2832/2832 [==============================] - 3s 884us/step - loss: 0.0330 - acc: 0.9880
Epoch 11/

In [0]:
from operator import truediv

def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc

In [58]:
test_cls = test_labels - 1
prediction = moment_lidar_model.predict(moment_lidar_test_patches).argmax(axis=-1)



from sklearn import metrics, preprocessing



overall_acc = metrics.accuracy_score(prediction, test_cls)
kappa = metrics.cohen_kappa_score(prediction, test_cls)
confusion_matrix = metrics.confusion_matrix(prediction, test_cls)
each_acc, average_acc = AA_andEachClassAccuracy(confusion_matrix) 


print("Overall Accuracy of training sapmles : ",overall_acc)
print("Average Accuracy of training samples : ",average_acc)
print("Kappa statistics of training samples : ",kappa)
print("Each class accuracy of training samples : ", each_acc)
print("Confusion matrix :\n", confusion_matrix)

Overall Accuracy of training sapmles :  0.4314175616954989
Average Accuracy of training samples :  0.44144128788590786
Kappa statistics of training samples :  0.38336568433788976
Each class accuracy of training samples :  [0.21666667 0.51787916 0.44674556 0.45859873 0.65605096 0.31564246
 0.50787402 0.34647739 0.22702278 0.53016688 0.84392419 0.37210584
 0.31746032 0.48177083 0.38323353]
Confusion matrix :
 [[156  51   0  17  78   0  53  70 102 137  11  36   3   1   5]
 [ 37 420  25  71  25   0  13  23  22  68   4  22  31   0  50]
 [ 23  24 151  12   1   0   0   0  13   0   0  40   1   0  73]
 [  0   4   0 288   9   0 106 132  45  16   0   5   8   2  13]
 [ 67   2   0   0 412   0  13   0 114   0   0   4  16   0   0]
 [ 46   0   0  67  13 113  12   4  19   0   6  69   9   0   0]
 [  2  17   1 163   4   0 516  29  93  49   0  33   0  16  93]
 [ 76 129   1 337 279   4 166 659  80  15   9  89   4   7  47]
 [230  77 237   9 106   2  77   0 289  88  10  46  51   2  49]
 [ 78  64   0   0  48 

In [0]:
moment_lidar_model.save_weights('Models/moment_lidar_model_weights.h5')